# ExoNAMD API v1.0


## Summary

The observer wants to compute the relative and/or absolute NAMD of:

- a given multiplanetary system;
- a subset of multiplanetary systems;
- all the known ones.

This tool handles all of the above cases.


In [77]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime
from datetime import timedelta
from spright import RMRelation
import warnings

from exonamd.utils import ROOT
from exonamd.utils import fetch_aliases
from exonamd.utils import update_host
from exonamd.utils import update_planet


warnings.filterwarnings("ignore")
pd.options.display.max_columns = 20
pd.options.display.max_rows = 30
pd.options.mode.copy_on_write = True

### Task 1: getting the data

This task retrieves the parameters of confirmed systems from the NASA Exoplanet Archive database, and stores them in a local database.


In [79]:
# Define the URL for the API
url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync"

# Define the date you want to filter by
from_scratch = False  # change to False in production
if from_scratch:
    latest = datetime.strptime("1990-01-01", "%Y-%m-%d")  # Example date
else:
    # df_old = pd.read_csv(os.path.join(ROOT, "data", "exo.csv"))
    df_old = pd.read_csv(os.path.join("../exonamd/", "data", "exo.csv"))
    latest = df_old["rowupdate"].max()
    latest = datetime.strptime(latest, "%Y-%m-%d")  # Example date
    latest = latest - timedelta(days=1)

# Convert the date to a string in the format 'YYYY-MM-DD'
latest = latest.strftime("%Y-%m-%d")

# Define the multiplicity you want to filter by
multiplicity = 5

# Define the SQL query to retrieve the required data
query = f"""
SELECT 
    hostname, 
    pl_name, 
    default_flag,
    rowupdate,
    sy_pnum, 
    st_rad,
    st_mass,
    pl_orbper,
    pl_orbsmax, 
    pl_orbsmaxerr1, 
    pl_orbsmaxerr2, 
    pl_rade,
    pl_radeerr1,
    pl_radeerr2,
    pl_bmasse, 
    pl_bmasseerr1, 
    pl_bmasseerr2, 
    pl_orbeccen, 
    pl_orbeccenerr1, 
    pl_orbeccenerr2, 
    pl_orbincl, 
    pl_orbinclerr1, 
    pl_orbinclerr2,
    pl_trueobliq,
    pl_trueobliqerr1,
    pl_trueobliqerr2,
    pl_ratdor,
    pl_ratror
FROM ps
WHERE
    sy_pnum > '{multiplicity}'
    AND rowupdate > '{latest}'
"""

# Define the parameters for the request
params = {
    "query": query,
    "format": "json",
}

# Make the request to the API
response = requests.get(url, params=params)

if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Convert the JSON data to a pandas DataFrame
    df = pd.DataFrame(data)
else:
    print(f"Error: {response.status_code}")

Deal with None values


In [80]:
df = df.replace({None: np.nan, "": np.nan})

### Task 2: dealing with the aliases


Fetch aliases


In [81]:
aliases = fetch_aliases(df["hostname"].unique())

Fetched 2/2 entries on try 1


Curate aliases


In [82]:
# # For testing
# df2 = df.copy()
# df2.loc[df2['hostname'] == 'Kepler-20', 'hostname'] = 'KOI-70'
# df2.loc[df2['hostname'] == 'Kepler-80', 'hostname'] = 'KIC 4852528'
# df2['hostname'] = df2.apply(update_host, args=(aliases, True), axis=1)
df["hostname"] = df.apply(update_host, args=(aliases, False), axis=1)

# # For testing
# df2 = df.copy()
# df2.loc[df2['pl_name'] == 'Kepler-20 c', 'pl_name'] = 'KOI-70.01'
# df2.loc[df2['pl_name'] == 'Kepler-11 b', 'pl_name'] = 'KOI-157 b'
# df2['pl_name'] = df2.apply(update_planet, args=(aliases, True), axis=1)
df["pl_name"] = df.apply(update_planet, args=(aliases, False), axis=1)

In [83]:
# Double check that the names are consistent

for hostname in df["hostname"].unique():
    df_host = df[df["hostname"] == hostname]
    names = df_host["pl_name"]
    if len(set([name[:3] for name in names])) > 1:
        print(f"Inconsistent name for {hostname}")

### Task 3: computing missing values (if any) from simple equations


In [84]:
from exonamd.utils import solve_a_rs
from exonamd.utils import solve_rprs
from exonamd.utils import solve_a_period


def complete_values(row):

    sma = row["pl_orbsmax"]
    ars = row["pl_ratdor"]
    rstar = row["st_rad"]
    rplanet = row["pl_rade"]
    rprs = row["pl_ratror"]
    period = row["pl_orbper"]
    mstar = row["st_mass"]

    # Rank groups
    a_rs_ = np.isnan(sma) + np.isnan(ars) + np.isnan(rstar)
    rprs_ = np.isnan(rplanet) + np.isnan(rprs) + np.isnan(rstar)
    a_period_ = np.isnan(period) + np.isnan(sma) + np.isnan(mstar)
    solve_order = np.argsort([a_rs_, rprs_, a_period_])
    for i in solve_order:
        if i == 0:
            # Solve semi-major axis -- stellar radius system of equations.
            solution = solve_a_rs(sma, rstar, ars)
            sma, rstar, ars = solution
        elif i == 1:
            # Solve planet radius -- stellar radius system of equations.
            solution = solve_rprs(rplanet, rstar, rprs)
            rplanet, rstar, rprs = solution
        elif i == 2:
            # Solve period-sma-mstar system of equations.
            solution = solve_a_period(period, sma, mstar)
            period, sma, mstar = solution

    return sma, ars, rstar, rplanet, rprs, period, mstar

In [85]:
df[
    [
        "pl_orbsmax",
        "pl_ratdor",
        "st_rad",
        "pl_rade",
        "pl_ratror",
        "pl_orbper",
        "st_mass",
    ]
] = df.apply(complete_values, axis=1, result_type="expand")

### Task 4: storing the curated database


In [86]:
if not from_scratch:
    df_new = df.copy()
    df = pd.concat([df_new, df_old], ignore_index=True)
    df = df.drop_duplicates(keep="last")

df.to_csv(os.path.join("../exonamd/", "data", "exo.csv"), index=False)

Drop columns that are no longer needed


In [87]:
df.drop(
    columns=[
        "pl_ratdor",
        "st_rad",
        "pl_ratror",
        "pl_orbper",
        "st_mass",
    ],
    inplace=True,
)

### Task 5: input missing values (if any) by interpolation


Use nanmedian to thin down the data


In [88]:
keep_indices = []

for planet in df["pl_name"].unique():
    df_planet = df[df["pl_name"] == planet]
    default_index = df_planet[df_planet["default_flag"] == 1].index
    for col in [
        c
        for c in df_planet.columns
        if c not in ["hostname", "pl_name", "default_flag", "rowupdate"]
    ]:
        avg = np.nanmedian(df_planet[col].values)
        df.loc[default_index, col] = avg
    keep_indices.extend(default_index)

df.drop(df.index[~df.index.isin(keep_indices)], inplace=True)
df.drop(columns="default_flag", inplace=True)

Instantiate flags


In [89]:
df["flag"] = "0"

Solve missing eccentricity values


In [90]:
def solve_eccentricity(row):
    ecc = row["pl_orbeccen"]
    eccerr1 = row["pl_orbeccenerr1"]
    eccerr2 = row["pl_orbeccenerr2"]
    flag = row["flag"]

    if np.isnan(ecc):
        ecc = 0.63 * row["sy_pnum"] ** (-1.02)
        eccerr1 = 0.0
        eccerr2 = 0.0
        flag += "1+-"

    if np.isnan(eccerr1):
        eccerr1 = 0.0
        flag += "1+"

    if np.isnan(eccerr2):
        eccerr2 = 0.0
        flag += "1-"

    return ecc, eccerr1, eccerr2, flag

In [91]:
df[
    [
        "pl_orbeccen",
        "pl_orbeccenerr1",
        "pl_orbeccenerr2",
        "flag",
    ]
] = df.apply(solve_eccentricity, axis=1, result_type="expand")

Solve missing planetary mass values


In [92]:
rmr = RMRelation()


def solve_mass(row, min_radius=0.5, max_radius=6.0):

    mass = row["pl_bmasse"]
    masserr1 = row["pl_bmasseerr1"]
    masserr2 = row["pl_bmasseerr2"]
    radius = row["pl_rade"]
    radiuserr1 = row["pl_radeerr1"]
    radiuserr2 = row["pl_radeerr2"]
    flag = row["flag"]

    if (
        np.isnan(mass)
        and radius > min_radius
        and radius < max_radius
        and not np.isnan(radiuserr1)
        and not np.isnan(radiuserr2)
    ):
        mds = rmr.predict_mass(radius=(radius, 0.5 * (radiuserr1 - radiuserr2)))
        q16, q50, q84 = np.percentile(mds.samples, [16, 50, 84])
        mass = q50
        masserr1 = q84 - q50
        masserr2 = q16 - q50
        flag += "2+-"

    if np.isnan(masserr1):
        masserr1 = 0.0
        flag += "2+"

    if np.isnan(masserr2):
        masserr2 = 0.0
        flag += "2-"

    return mass, masserr1, masserr2, flag

In [93]:
df[
    [
        "pl_bmasse",
        "pl_bmasseerr1",
        "pl_bmasseerr2",
        "flag",
    ]
] = df.apply(solve_mass, axis=1, result_type="expand")

Drop columns that are no longer needed


In [94]:
df = df.drop(columns=["pl_rade", "pl_radeerr1", "pl_radeerr2"])

Remove systems where at least one planet has no mass or semi-major axis (if any)


In [95]:
remove_indices = []
for hostname in df["hostname"].unique():
    df_host = df[df["hostname"] == hostname]
    if df_host[["pl_bmasse", "pl_orbsmax"]].isnull().any().any():
        print(
            f"System {hostname} will be removed"
            "due to missing values in mass or semi-major axis."
        )
        remove_indices.extend(df_host.index)
df.drop(df.index[remove_indices], inplace=True)

Solve missing values in inclinations


In [96]:
def solve_inclination(row, df):

    hostname = row["hostname"]
    incl = row["pl_orbincl"]
    inclerr1 = row["pl_orbinclerr1"]
    inclerr2 = row["pl_orbinclerr2"]
    flag = row["flag"]

    host = df[df["hostname"] == hostname]
    mass_max = host["pl_bmasse"].idxmax()
    inclnan = host[host["pl_orbincl"].isnull()]

    # if inclination is not nan, check the errors
    # if they are nan, set them to 0
    if not np.isnan(incl):
        if np.isnan(inclerr1):
            inclerr1 = 0.0
            flag += "3+"
        if np.isnan(inclerr2):
            inclerr2 = 0.0
            flag += "3-"

    elif len(inclnan) == len(host):  
        # i.e., all inclinations in the system are nan
        incl = 90.0
        inclerr1 = 0.0
        inclerr2 = 0.0
        flag += "3+-"

    elif mass_max in inclnan.index:
        # i.e., the most massive planet has inclination nan
        # Note: we could change this to the next most massive planet later on
        incl = 90.0
        inclerr1 = 0.0
        inclerr2 = 0.0
        flag += "3+-"

    else:
        # i.e., the most massive planet reports inclination
        incl = host.loc[mass_max, "pl_orbincl"]
        inclerr1 = host.loc[mass_max, "pl_orbinclerr1"]
        inclerr2 = host.loc[mass_max, "pl_orbinclerr2"]
        flag += "3+-"

    return incl, inclerr1, inclerr2, flag

In [97]:
df[
    [
        "pl_orbincl",
        "pl_orbinclerr1",
        "pl_orbinclerr2",
        "flag",
    ]
] = df.apply(solve_inclination, args=(df,), axis=1, result_type="expand")

Solve missing values in semi-major axis uncertainties


In [98]:
def solve_sma(row):

    smaerr1 = row["pl_orbsmaxerr1"]
    smaerr2 = row["pl_orbsmaxerr2"]
    flag = row["flag"]

    if np.isnan(smaerr1):
        smaerr1 = 0.0
        flag += "4+"

    if np.isnan(smaerr2):
        smaerr2 = 0.0
        flag += "4-"

    return smaerr1, smaerr2, flag

In [99]:
df[
    [
        "pl_orbsmaxerr1",
        "pl_orbsmaxerr2",
        "flag",
    ]
] = df.apply(solve_sma, axis=1, result_type="expand")

### Task 6: storing the curated+interpolated database


In [101]:
df.to_csv(os.path.join("../exonamd/", "data", "exo_interp.csv"), index=False)